In [6]:
using CSV
using DataFrames
using Dates
using LinearAlgebra
using Plots
using Statistics

In [7]:
df_path= joinpath("current.csv")
df = CSV.read(df_path, DataFrame)

Row,sasdate,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,ACOGNO,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,⋯
,String15,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64?,Float64?,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64?,Float64,Float64?,Float64?,Float64,Float64,Float64,Int64,Float64,Int64,Float64,Float64,Float64?,Float64?,Float64,Float64?,Float64?,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,⋯
1,Transform:,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,2,2.0,5,5,2.0,2.0,5,5,5,5,5,5,5,5,5.0,5,5,5,5,5,5,5.0,5.0,5,5,1.0,2.0,1.0,4,4,4,4,4,4,4,4,4,4,5,5.0,5.0,5.0,5.0,2.0,6.0,6.0,5.0,6,6.0,7,6.0,6.0,6.0,2.0,5.0,2.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,5.0,6.0,⋯
2,1/1/1959,2583.56,2426.0,15.188,2.76677e5,18235.8,21.9665,23.3891,22.2688,31.7011,19.0149,38.1501,8.0434,20.0387,11.9952,30.6591,20.8382,19.9173,34.6654,80.1973,1357,0.333579,67936,63868,6.0,16.3,1574,1169,1396,594,802,291078,52478,18796,713.0,2993,14998,8740,6258,33682,10774,2568.7,5350.3,2418,8105,39.8,2.5,40.2,1657,350,452,505,350,missing,missing,missing,missing,missing,missing,14716.5,missing,42620.3,84889.6,1.56,138.9,286.6,987.9,50500,18.9,18300,35.213,24.9242,48.9612,0.124965,55.62,3.15834,18.4457,2.48,3.3,2.82,3.09,3.36,4.01,4.02,4.12,4.87,0.82,0.34,0.61,0.88,1.53,1.54,1.64,2.39,missing,4.3122,359.842,2.8065,0.9671,33.1,⋯
3,2/1/1959,2593.6,2434.8,15.346,278714.0,18369.6,22.3966,23.7048,22.4617,31.9337,19.1147,38.5147,8.151,20.6733,12.5477,31.1959,21.2213,19.8631,34.1932,81.4428,1421,0.358386,67649,63684,5.9,15.5,1554,1164,1277,545,732,282958,52688,18890,704.2,2980,15115,8839,6276,33798,10816,2575.4,5381.3,2420,8116,39.7,2.6,40.3,1667,346,469,508,344,missing,missing,missing,missing,missing,missing,15400.2,missing,43677.2,85181.8,1.53934,139.4,287.7,992.1,49800,18.6,18100,35.2201,25.227,49.5137,0.125765,54.77,3.21952,18.4181,2.43,3.26,2.7,3.13,3.54,3.96,3.96,4.14,4.89,0.83,0.27,0.7,1.11,1.53,1.53,1.71,2.46,missing,4.3133,359.842,2.8093,0.9748,33.2,⋯
4,3/1/1959,2610.4,2452.7,15.491,2.77775e5,18523.1,22.7193,23.8483,22.5719,31.9337,19.489,38.3324,8.2391,21.2103,13.042,31.6645,21.5533,19.9986,35.0589,82.4769,1524,0.400947,68068,64267,5.6,15.3,1459,1093,1210,530,680,260346,53014,19069,704.1,3013,15259,8965,6294,33945,10873,2584.4,5431.6,2430,8132,40.0,2.8,40.4,1620,330,413,503,374,missing,missing,missing,missing,missing,missing,15745.4,missing,44781.6,85620.2,1.52901,139.7,289.2,998.3,49700,18.4,17800,35.1304,25.4218,50.0077,0.126123,56.16,3.15171,18.9994,2.8,3.35,2.8,3.13,3.61,3.99,3.99,4.13,4.85,0.55,0.0,0.33,0.81,1.19,1.19,1.33,2.05,missing,4.3228,359.842,2.8127,0.9698,33.2,⋯
5,4/1/1959,2627.45,2470.0,15.435,2.83363e5,18534.5,23.2032,24.1927,22.9026,32.4374,19.6138,38.9705,8.4544,21.7961,13.4346,32.6452,22.013,20.0799,34.9015,83.9922,1589,0.44

In [8]:
### Clean the DataFrame by removing the row with transformation codes
df_cleaned = df[2:end, :]
date_format = "mm/dd/yyyy"
df_cleaned[!, :sasdate] = Dates.Date.(df_cleaned[!, :sasdate], date_format)

df_original = copy(df_cleaned)
df_cleaned = coalesce.(df_cleaned, NaN)
df_original = coalesce.(df_original, NaN)
### Create a DataFrame with the transformation codes
transformation_codes = DataFrame(Series = names(df)[2:end],  Transformation_Code = collect(df[1, 2:end]))

Row,Series,Transformation_Code
,String,Real
1,RPI,5.0
2,W875RX1,5.0
3,DPCERA3M086SBEA,5.0
4,CMRMTSPLx,5.0
5,RETAILx,5.0
6,INDPRO,5.0
7,IPFPNSS,5.0
8,IPFINAL,5.0
9,IPCONGD,5.0


In [9]:
### Function to apply transformations based on the transformation code
function apply_transformation(series, code)
    if code == 1
        # No transformation
        return series
    elseif code == 2
        # First difference
        return mdiff(series)
    elseif code == 3
        # Second difference
        return mdiff(mdiff(series))
    elseif code == 4
        # Log
        return log.(series)
    elseif code == 5
        # First difference of log
        return mdiff(log.(series))
    elseif code == 6
        # Second difference of log
        return mdiff(mdiff(log.(series)))
    elseif code == 7
        # Delta (x_t/x_{t-1} - 1)
        return series ./ lag(series, 1) .- 1
    else
        throw(ArgumentError("Invalid transformation code"))
    end
end

apply_transformation (generic function with 1 method)

In [10]:
### Helper function to lag a series (since Julia doesn't have a built-in function to lag)
function lag(v::Vector, l::Integer)
    nan = [NaN for _ in 1:l]
    return [nan; v[1:(end-l)]]
end

function lead(v::Vector, l::Integer)
    nan = [NaN for _ in 1:l]
    return [v[(l+1):end]; nan]
end

### mdiff function to calculate the first difference of a series
function mdiff(v::Vector)
    return v .- lag(v, 1)
end

### Applying the transformations to each column in df_cleaned based on transformation_codes
for row in eachrow(transformation_codes)
    series_name = Symbol(row[:Series])
    code = row[:Transformation_Code]
    @show series_name, code
    df_cleaned[!, series_name] = apply_transformation(df_cleaned[!, series_name], code)
end

(series_name, code) = (:RPI, 5.0)
(series_name, code) = (:W875RX1, 5.0)
(series_name, code) = (:DPCERA3M086SBEA, 5.0)
(series_name, code) = (:CMRMTSPLx, 5.0)
(series_name, code) = (:RETAILx, 5.0)
(series_name, code) = (:INDPRO, 5.0)
(series_name, code) = (:IPFPNSS, 5.0)
(series_name, code) = (:IPFINAL, 5.0)
(series_name, code) = (:IPCONGD, 5.0)
(series_name, code) = (:IPDCONGD, 5.0)
(series_name, code) = (:IPNCONGD, 5.0)
(series_name, code) = (:IPBUSEQ, 5.0)
(series_name, code) = (:IPMAT, 5.0)
(series_name, code) = (:IPDMAT, 5.0)
(series_name, code) = (:IPNMAT, 5.0)
(series_name, code) = (:IPMANSICS, 5.0)
(series_name, code) = (:IPB51222S, 5.0)
(series_name, code) = (:IPFUELS, 5.0)
(series_name, code) = (:CUMFNS, 2.0)
(series_name, code) = (:HWI, 2)
(series_name, code) = (:HWIURATIO, 2.0)
(series_name, code) = (:CLF16OV, 5)
(series_name, code) = (:CE16OV, 5)
(series_name, code) = (:UNRATE, 2.0)
(series_name, code) = (:UEMPMEAN, 2.0)
(series_name, code) = (:UEMPLT5, 5)
(series_name, cod

In [11]:
### The transformation create missing values at the top, so let's remove them
df_cleaned = df_cleaned[3:end, :]

function compute_forecast(target = :INDPRO, x_var = [:CPIAUCSL, :FEDFUNDS],H = [1 4 8],p = 4)
    y_hat=[]

    y_raw = df_cleaned[!, target]
    x_raw = select(df_cleaned, x_var)    

    Y_lagged = hcat([lag(y_raw, p) for i in 1:p]...)
    X_lagged = hcat([lag(x_raw[:, j], i) for j in 1:size(x_raw, 2) for i in 1:p]...)
    
    X = hcat(ones(size(Y_lagged, 1)), Y_lagged, X_lagged)
  
    X_T = X[end, :]
    
    for h in H
        y_h = lead(y_raw, h)
        y  = y_h[p+1:(end-h)]
        X_ = X[p+1:(end-h), :]
        beta_ols = X_ \ y
        forecast = (X_T' * beta_ols) * 100
        push!(y_hat,forecast)  
    end
    return y_hat
end

compute_forecast (generic function with 5 methods)

In [12]:
# CREATE THE FORECASTING FUNCTION
function compute_error(target = :INDPRO, x_var = [:CPIAUCSL, :FEDFUNDS],H = [1 4 8],p = 4, end_date = Dates.Date("12/01/1999", date_format))
    y_hat = []
    y_actual = []
    rt_df = filter(row -> row[:sasdate] <= end_date, df_cleaned)
    

    for h in H
        os = end_date + Dates.Month(h)
        push!(y_actual, (filter(row -> row[:sasdate] == os, df_cleaned)))
    end

    y_raw = rt_df[!, target]
    x_raw = select(rt_df, x_var)    

    Y_lagged = hcat([lag(y_raw, p) for i in 1:p]...)
    X_lagged = hcat([lag(x_raw[:, j], i) for j in 1:size(x_raw, 2) for i in 1:p]...)
    
    X = hcat(ones(size(Y_lagged, 1)), Y_lagged, X_lagged)
  
    X_T = X[end, :]
    
    for h in H
        y_h = lead(y_raw, h)
        y  = y_h[p+1:(end-h)]
        X_ = X[p+1:(end-h), :]
        beta_ols = X_ \ y
        forecast = (X_T' * beta_ols) * 100
        push!(y_hat,forecast)  
    end
    
    y_actual_matrix = vcat(y_actual[1],y_actual[2],y_actual[3])
    error_value = y_hat - y_actual_matrix[!,target] 
    error_value = hcat(error_value)
    
    return error_value
    

end

compute_error (generic function with 6 methods)

In [13]:
# CREATE THE ERROR-COMPUTING FUNCTION
function evaluate_model(target = :INDPRO, x_var = [:CPIAUCSL, :FEDFUNDS],H = [1 4 8],p = 4, t0 = "12/01/1999")

    t0 = Dates.Date(t0,date_format)
    e = []
    T = []
   

    for j in 1:10
        t0 = t0 + Dates.Month(1)
        ehat = compute_error(target,x_var,H,p, t0)
        ehat = ehat'
        push!(e,ehat)
        push!(T,t0)
    end
    a = vcat(e...)
    sqrd_a = a .^ 2
    mean_sqrd_a = (mean(sqrd_a, dims=1)) .^(0.5)
    return  mean_sqrd_a

end


evaluate_model (generic function with 6 methods)

In [14]:

# COMPUTING THE 3 DIFFERENT REGRESSIONS
forecast_INDPRO = compute_forecast(:INDPRO, [:CPIAUCSL, :TB3MS, :AWHMAN])
println(forecast_INDPRO)
forecast_CPI= compute_forecast(:CPIAUCSL, [:M1SL, :UNRATE, :FEDFUNDS, :OILPRICEx])
println(forecast_CPI)
forecast_FF = compute_forecast(:FEDFUNDS, [:CPIAUCSL, :UNRATE, :INDPRO, :TB3MS, :INVEST, :DTCTHFNM, :NONREVSL, :PCEPI])
println(forecast_FF)

regressione1 = evaluate_model(:INDPRO, [:CPIAUCSL, :TB3MS, :AWHMAN])
println(regressione1)
regressione2 = evaluate_model(:CPIAUCSL, [:M1SL, :UNRATE, :FEDFUNDS, :OILPRICEx])
println(regressione2)
regressione3 = evaluate_model(:FEDFUNDS, [:CPIAUCSL, :UNRATE, :INDPRO, :TB3MS, :INVEST, :DTCTHFNM, :NONREVSL, :PCEPI])
println(regressione3)

Any[0.2425919059177481, 0.2912798088947899, 0.3252145273681714]
Any[-0.05643642398835013, 0.016930926642984335, 0.024599526152021916]
Any[8.225458821562086, -9.061173052372075, 26.14034411702213]
[0.3756036287054008 0.2931728496467193 0.25259659198416234]
[0.054932098090677735 0.07494818306843783 0.042703124051111194]
[6.588976477676038 6.399557205697552 10.29328847434379]
